In [25]:
import time
import json
import math
import numpy as np
import sympy as sp
from itertools import combinations
import chipsplitting as cs
from chipsplitting.hyperfield import (
    HyperfieldPascalForm as HVPascal, 
    HyperfieldHomogeneousLinearSystem as HVLinearSystem,
    HyperfieldVector
)
from chipsplitting import pairing_matrix, print_matrix, PascalForm

## Utils

In [26]:
# utils
def save_fundamental_models(model, json_file_name):
    with open(json_file_name, 'w') as json_file:
        json.dump([{"support": support, "solution": [x for x in solution] } for support, solution in model], json_file, default=float)
    
    print(f"JSON dump has been written to {json_file_name}")

def load_fundamental_models(json_file_name):
    with open(json_file_name, 'r') as json_file:
        data = json.load(json_file)
        return [(item["support"], item["solution"]) for item in data]

def expand_expression(expression):
    if expression.startswith('t^'):
        power = int(expression[2:])
        return [1 if k == power else 0 for k in range(power + 1)]
    elif expression.startswith('(1-t)^'):
        power = int(expression[len('(1-t)^'):])
        return [math.comb(power,k) * (-1)**(k) for k in range(power + 1)]
    else:
        raise ValueError("Invalid expression")

def multiply_expression(left_exp: list[int], right_exp: list[int]):
    left_power = len(left_exp) - 1
    right_power = len(right_exp) - 1
    degree = left_power + right_power
    t_coefficients = [0] * (degree + 1)
    for a in range(left_power + 1):
        for b in range(right_power + 1):
            t_coefficients[a + b] += left_exp[a] * right_exp[b]
    return t_coefficients

def pad_with_zero(list, desired_length: int):
    if len(list) < desired_length:
        return list + [0] * (desired_length - len(list))
    return list

def create_matrix_from_support(degree: int, support: list[int]):
    support = [utils.to_coordinate(index) for index in support]
    A = np.array([pad_with_zero(multiply_expression(expand_expression(f"t^{col}"), expand_expression(f"(1-t)^{row}")), degree + 1) for col, row in support])
    return A.T

def is_fundamental(support_as_index, degree):
    A = create_matrix_from_support(degree, support_as_index)
    b = np.array([1] + [0] * degree)
    sol, norm, rank, singular_values = np.linalg.lstsq(A, b, rcond=None)

    if rank < len(support_as_index):
        return None

    if len(norm) == 0 or np.isclose(norm[0], 0):
        return sol
    
    return None

## Compute possible supports of fundamental models

Iterating through all possible supports is not feasible. Luckily, we can exclude invalid supports using the invertibility criterion.

In [27]:
def find_positive_supports(pos_support_size, mode = "fast", expand = False):
    degree_end = 2 * pos_support_size - 3
    # hyperfield variety generated by all pascal equations
    S = {}
    for d in range(1, degree_end + 1):
        start = time.time()
        num_cells = cs.utils.gauss(d+1)
        # positive cells are all cells except the (0,0) cell
        num_pos_cells = num_cells - 1
        # skip if the degree has not enough cells
        if num_pos_cells < pos_support_size:
            continue
            
        base_types = ["diag", "row", "col"]
        A = [HVPascal(d, b, k) for b in base_types for k in range(d + 1)]
        linear_system = HVLinearSystem(A)
        
        # I believe quick_solve_loop has a bug
        if mode == "fast":
            solutions = linear_system.quick_solve_loop(pos_support_size)
        else:
            solutions = linear_system.quick_solve(pos_support_size)
        
        if expand:
            def expand_solution(sol):
                num_selections = pos_support_size - len(sol)
                domain = [x for x in range(cs.utils.gauss(d + 1)) if x not in sol]
                return [tuple(sorted(sol + c)) for c in combinations(domain, num_selections)]

            expanded = []
            for sol in solutions:
                if len(sol) < pos_support_size:
                    expanded += expand_solution(sol)
                else:
                    expanded.append(sol)
            solutions = list(set(expanded))
        
        S[d] = solutions
        print(f"d={d}, #supports={len(solutions)}. Elapsed: {time.time() - start}")
    return S

In [29]:
possible_supports = {}
n_begin = 6
n_end = 6
for pos_support_size in range(n_begin + 1, n_end + 2):
    print(f"n={pos_support_size - 1}")
    possible_supports[pos_support_size] = find_positive_supports(pos_support_size, mode = "fast", expand = True)
print("Reducing possible solutions by applying symmetry...")

possible_supports_copy = {}

for n, d_to_solutions in possible_supports.items():
    d_to_solutions_copy = {}
    for d, solutions in d_to_solutions.items():
        solutions_copy = set()
        for support in solutions:
            if tuple(sorted(cs.reflect_support(support))) not in solutions_copy:
                solutions_copy.add(support)
        d_to_solutions_copy[d] = list(solutions_copy)
    possible_supports_copy[n] = d_to_solutions_copy

possible_supports = possible_supports_copy

for n, d_to_solutions in possible_supports.items():
    print("")
    print(f"n={n-1}")
    for d, solutions in d_to_solutions.items():
        print(f"d={d}, #supports={len(solutions)}")

n=6
d=3, #supports=89. Elapsed: 0.002763032913208008
d=4, #supports=2431. Elapsed: 0.5797679424285889
d=5, #supports=21136. Elapsed: 0.05563497543334961
d=6, #supports=104439. Elapsed: 0.23417878150939941
d=7, #supports=360471. Elapsed: 0.8066580295562744
d=8, #supports=982891. Elapsed: 6.267210960388184
d=9, #supports=2319165. Elapsed: 5.845315933227539
d=10, #supports=4854071. Elapsed: 12.536600112915039
d=11, #supports=9418679. Elapsed: 37.33899688720703
Reducing possible solutions by applying symmetry...

n=6
d=3, #supports=48
d=4, #supports=1226
d=5, #supports=10635
d=6, #supports=52284
d=7, #supports=180527
d=8, #supports=491601
d=9, #supports=1160287
d=10, #supports=2427353
d=11, #supports=4710754


In [30]:
json_file_name = "section8_n6_possible_supports.json"
with open(json_file_name, 'w') as json_file:
    json.dump(possible_supports, json_file, default=int)

print(f"JSON dump has been written to {json_file_name}")

JSON dump has been written to section8_n6_possible_supports.json


In [5]:
json_file_name = "section8_n6_possible_supports.json"
with open(json_file_name, 'r') as json_file:
    possible_supports = json.load(json_file)
    possible_supports = {int(key) : {int(d) : value for d, value in degree_to_support.items()} for key, degree_to_support in possible_supports.items()}

print(f"JSON has been loaded")

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x108a4e7c0>>
Traceback (most recent call last):
  File "/Users/duc/.pyenv/versions/3.9.18/envs/master-thesis/lib/python3.9/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


JSON has been loaded


## Compute fundamental models for $n=6$

### Code

In [58]:
def is_positive(solution):
    for x in solution:
        if np.isclose(x,0):
            return False
        if x < 0:
            return False
    return True

def is_parametric(solution):
    for x in solution:
        if not x.is_number:
            return True
    return False

def is_asymmetric(support):
    return list(sorted(support)) != list(sorted(apply_symmetry(support)))

def filter_fundamental_models(fundamental_models):
    return [(support, solution) for support, solution in fundamental_models if solution and is_positive(solution)]

def find_fundamental_models(n, degrees):
    if isinstance(degrees, int):
        degrees = [degrees]
        
    fundamental_models = []
    
    for d in degrees:
        start = time.time()
        supports_to_check = possible_supports[n + 1][d]

        print("")
        print(f"d={d}: {len(supports_to_check)} supports to check")

        for index, support in enumerate(supports_to_check):
            if index == 100:
                elapsed = time.time() - start
                print(f"Solved linear equations of 100 supports in {round(elapsed, 2)}s. Estimate: {round(len(supports_to_check) / 100 * elapsed / 60, 2)}min")
                
            solution = is_fundamental(support, d)
            fundamental_models.append((support, solution))

        end = time.time()
        print(f"Solved linear equations for all supports. Elapsed time: {round(end - start, 2)}s")

    print()
    
    fundamental_models = [(support, solution) for support, solution in fundamental_models if solution is not None]
    print(f"After excluding supports with no solutions, {len(fundamental_models)} fundamental models are left.")

    #fundamental_models = [(support, solution) for support, solution in fundamental_models if not is_parametric(solution)]
    #print(f"After excluding supports with parametric solutions, {len(fundamental_models)} fundamental models are left.")

    fundamental_models = [(support, solution) for support, solution in fundamental_models if is_positive(solution)]
    print(f"After excluding supports with non-positive solutions, {len(fundamental_models)} fundamental models are left.")

    print(f"")
    print(f"We have found {len(fundamental_models)} fundamental models.")

    missing_models_from_symmetry = [(apply_symmetry(support), solution) for support, solution in fundamental_models if is_asymmetric(support)]
    print(f"Found additional {len(missing_models_from_symmetry)} fundamental models due to symmetry.")
    
    return fundamental_models + missing_models_from_symmetry

### $d=3,4,5$

In [46]:
fundamental_models = find_fundamental_models(6, [3,4,5])
print()
print(f"There exist exactly {len(fundamental_models)} fundamental models ✨")


d=3: 48 supports to check
Solved linear equations for all supports. Elapsed time: 0.01s

d=4: 1226 supports to check
Solved linear equations of 100 supports in 0.01s. Estimate: 0.0min
Solved linear equations for all supports. Elapsed time: 0.08s

d=5: 10635 supports to check
Solved linear equations of 100 supports in 0.0s. Estimate: 0.01min
Solved linear equations for all supports. Elapsed time: 0.45s

After excluding supports with no solutions, 0 fundamental models are left.
After excluding supports with parametric solutions, 0 fundamental models are left.
After excluding supports with non-positive solutions, 0 fundamental models are left.

We have found 0 fundamental models.
Found additional 0 fundamental models due to symmetry.

There exist exactly 0 fundamental models ✨


### $d=6$

In [59]:
n = 6
d = 6
fundamental_models = find_fundamental_models(n, d)
print()
print(f"There exist exactly {len(fundamental_models)} fundamental models ✨")


d=6: 52284 supports to check
Solved linear equations of 100 supports in 0.02s. Estimate: 0.16min
Solved linear equations for all supports. Elapsed time: 2.44s

After excluding supports with no solutions, 41359 fundamental models are left.
After excluding supports with non-positive solutions, 3370 fundamental models are left.

We have found 3370 fundamental models.
Found additional 3340 fundamental models due to symmetry.

There exist exactly 6710 fundamental models ✨


#### Save model and load model

In [40]:
save_fundamental_models(fundamental_models, f"section8_n{n}_d{d}.json")

JSON dump has been written to section8_n6_d6.json


In [6]:
# Run this cell if you want to load all the fundamental models for n=6 and d=6
fundamental_models = load_fundamental_models(f"section8_n6_d6.json")

### $d=7$

In [60]:
n = 6
d = 7
fundamental_models = find_fundamental_models(n, d)
print()
print(f"There exist exactly {len(fundamental_models)} fundamental models ✨")


d=7: 180527 supports to check
Solved linear equations of 100 supports in 0.02s. Estimate: 0.65min
Solved linear equations for all supports. Elapsed time: 11.99s

After excluding supports with no solutions, 27446 fundamental models are left.
After excluding supports with non-positive solutions, 1250 fundamental models are left.

We have found 1250 fundamental models.
Found additional 1171 fundamental models due to symmetry.

There exist exactly 2421 fundamental models ✨


#### Save model and load model

In [63]:
save_fundamental_models(fundamental_models, f"section8_n{n}_d{d}.json")

JSON dump has been written to section8_n6_d8.json


In [47]:
# Run this cell if you want to load all the fundamental models for n=6 and d=7
# fundamental_models = load_fundamental_models(f"section8_n6_d7.json")

### $d=8$

In [65]:
n = 6
d = 8
fundamental_models = find_fundamental_models(n, d)
print()
print(f"There exist exactly {len(fundamental_models)} fundamental models ✨")


d=8: 491601 supports to check
Solved linear equations of 100 supports in 0.02s. Estimate: 2.04min
Solved linear equations for all supports. Elapsed time: 31.64s

After excluding supports with no solutions, 31508 fundamental models are left.
After excluding supports with non-positive solutions, 324 fundamental models are left.

We have found 324 fundamental models.
Found additional 319 fundamental models due to symmetry.

There exist exactly 643 fundamental models ✨


In [66]:
save_fundamental_models(fundamental_models, f"section8_n{n}_d{d}.json")

JSON dump has been written to section8_n6_d8.json


### $d = 9$

In [67]:
n = 6
d = 9
fundamental_models = find_fundamental_models(n, d)
print()
print(f"There exist exactly {len(fundamental_models)} fundamental models ✨")


d=9: 1160287 supports to check
Solved linear equations of 100 supports in 0.01s. Estimate: 1.47min
Solved linear equations for all supports. Elapsed time: 79.31s

After excluding supports with no solutions, 47093 fundamental models are left.
After excluding supports with non-positive solutions, 106 fundamental models are left.

We have found 106 fundamental models.
Found additional 92 fundamental models due to symmetry.

There exist exactly 198 fundamental models ✨


In [68]:
save_fundamental_models(fundamental_models, f"section8_n{n}_d{d}.json")

JSON dump has been written to section8_n6_d9.json


### $d = 10$

In [69]:
n = 6
d = 10
fundamental_models = find_fundamental_models(n, d)
print()
print(f"There exist exactly {len(fundamental_models)} fundamental models ✨")


d=10: 2427353 supports to check
Solved linear equations of 100 supports in 0.03s. Estimate: 11.45min
Solved linear equations for all supports. Elapsed time: 183.01s

After excluding supports with no solutions, 68417 fundamental models are left.
After excluding supports with non-positive solutions, 16 fundamental models are left.

We have found 16 fundamental models.
Found additional 16 fundamental models due to symmetry.

There exist exactly 32 fundamental models ✨


In [70]:
save_fundamental_models(fundamental_models, f"section8_n{n}_d{d}.json")

JSON dump has been written to section8_n6_d10.json


### $d = 11$

In [71]:
n = 6
d = 11
fundamental_models = find_fundamental_models(n, d)
print()
print(f"There exist exactly {len(fundamental_models)} fundamental models ✨")


d=11: 4710754 supports to check
Solved linear equations of 100 supports in 0.03s. Estimate: 20.01min
Solved linear equations for all supports. Elapsed time: 347.83s

After excluding supports with no solutions, 99262 fundamental models are left.
After excluding supports with non-positive solutions, 2 fundamental models are left.

We have found 2 fundamental models.
Found additional 2 fundamental models due to symmetry.

There exist exactly 4 fundamental models ✨


In [72]:
save_fundamental_models(fundamental_models, f"section8_n{n}_d{d}.json")

JSON dump has been written to section8_n6_d11.json


## Result

```
| n \ d   |  1  |  2  |  3  |  4  |  5  |  6  |  7  |  8  |  9  |  10  |  11  |
| ------- | ----------------------------------------------------------------- |
| 1       |  1  |     |     |     |     |     |     |     |     |      |      |
| 2       |     |  3  |  1  |     |     |     |     |     |     |      |      |
| 3       |     |     | 12  |  4  |  2  |     |     |     |     |      |      |
| 4       |     |     |     | 82  | 38  |  10 |  4  |     |     |      |      |
| 5       |     |     |     |     | 602 | 254 |  88 |  24 |  2  |      |      |
| 6       |     |     |     |     |     | 6710| 2421| 643 | 198 |  32  |  4   |
```